In [1]:
import pandas as pd

# Population Data

In [2]:
key = ['Pro_Name','Pro_Code','Bgy_Name','Bgy_Code']

psgc = pd.read_csv('PSGC_Pro_Bgy.csv')

In [3]:
bgy_df = pd.read_excel('data/Population 2010, 2015 ARMM,BCT.xlsx')
drop_keys = ['REGI', 'PSGC_REGI','MuniCities','PSGC_CITY/MUNI','BCT']
bgy_df = bgy_df.drop(drop_keys,axis=1)
bgy_df = bgy_df.fillna(0)
rename_dict = {'PROV':'Pro_Name','PSGC_PROV':'Pro_Code','BrgyName':'Bgy_Name','PSGC_BRGY':'Bgy_Code'}
bgy_df = bgy_df.rename(columns=rename_dict)
bgy_df.head()

,Pro_Name,Pro_Code,Bgy_Name,Bgy_Code,Pop2010,Pop2015,HH_pop,num_hh0,num_hh0_mal,num_hh0_fem,...,80_abv_mal,num_hh0_fem.1,bel_20_hh0_fem,20-29_hh0_fem,30-39_hh0_fem,40-49_hh0_fem,50-59_hh0_fem,60-69_hh0_fem,70-79_hh0_fem,80_abv_fem
0,Basilan,PH150700000,Arco,PH150702001,800,672,667.0,160.0,137.0,23.0,...,3.0,23.0,0.0,1.0,3.0,0.0,8.0,6.0,2.0,3.0
1,Basilan,PH150700000,Ba-as,PH150702002,1266,1255,1255.0,231.0,159.0,72.0,...,3.0,72.0,0.0,9.0,13.0,16.0,13.0,14.0,7.0,0.0
2,Basilan,PH150700000,Baimbing,PH150702004,451,500,500.0,101.0,76.0,25.0,...,0.0,25.0,0.0,4.0,1.0,3.0,4.0,8.0,4.0,1.0
3,Basilan,PH150700000,Balagtasan,PH150702005,2597,2359,2359.0,434.0,358.0,76.0,...,6.0,76.0,1.0,2.0,10.0,18.0,24.0,7.0,13.0,1.0
4,Basilan,PH150700000,Balas,PH150702006,2809,2918,2918.0,546.0,380.0,166.0,...,2.0,166.0,2.0,26.0,39.0,28.0,32.0,28.0,7.0,4.0


In [4]:
pop_cols = [col for col in bgy_df.columns if ('num' in col.lower()) | ('pop' in col.lower()) | ('tot' in col.lower())]
bgy_df.columns = bgy_df.columns.str.replace('80_abv','80_abv_hh0')
sin_cols = [col for col in bgy_df.columns if ('sin_age' in col.lower()) & ('tot' not in col.lower())]

In [5]:
keys = key+sin_cols
sas_df = pd.melt(bgy_df[keys],id_vars=key)
sex_var_map = {'bot':'both','fem':'female','mal':'male'}
sas_df['variable'] = sas_df['variable'].str.replace('sin_age_','')
sas_df['sex'] = sas_df['variable'].str.split('_').str[0].replace(sex_var_map)
age_var_map = {'080_and_ove':'081','und_001':'000'}
sas_df['age'] = sas_df['variable'].str.replace('bot_|fem_|mal_','').replace(age_var_map)
sas_df['age'] = sas_df['age'].astype(int)
for start,end in zip(range(0,81,5),range(5,86,5)):
    cond = (sas_df['age']>=start) & (sas_df['age']<=end-1)
    sas_df.loc[cond,'age group'] = f'{start}-{end-1}'
sas_df = sas_df.drop(['variable','age'],axis=1)
keys = key+['age group','sex']
sas_df = sas_df.groupby(keys,as_index=False).sum()
age_grp_map = {'0-4':'below 5','80-84':'80 above'}
sas_df['age group'] = sas_df['age group'].replace(age_grp_map)
sas_df.columns = sas_df.columns.str.title()
sas_df.head()

,Pro_Name,Pro_Code,Bgy_Name,Bgy_Code,Age Group,Sex,Value
0,Basilan,PH150700000,Abong-Abong,PH150704001,below 5,both,149.0
1,Basilan,PH150700000,Abong-Abong,PH150704001,below 5,female,74.0
2,Basilan,PH150700000,Abong-Abong,PH150704001,below 5,male,75.0
3,Basilan,PH150700000,Abong-Abong,PH150704001,10-14,both,122.0
4,Basilan,PH150700000,Abong-Abong,PH150704001,10-14,female,68.0


In [6]:
sas_df.to_csv(f'output/age_sex_structure.csv',index=False)

# Education

In [31]:
educ_df = pd.read_excel('data/Number of schools per brgy.xlsx')
educ_df = educ_df.rename(columns={'BRGY':'Bgy_Name','PSGC':'Bgy_Code'})
educ_df = educ_df.merge(psgc,on=['Bgy_Name','Bgy_Code'],how='left')
educ_df.head()

,Bgy_Name,Bgy_Code,Number of School,Pro_Name,Pro_Code
0,Arco,PH150702001,1,Basilan,PH150700000
1,Ba-as,PH150702002,1,Basilan,PH150700000
2,Balagtasan,PH150702005,1,Basilan,PH150700000
3,Balas,PH150702006,1,Basilan,PH150700000
4,Balobo,PH150702007,1,Basilan,PH150700000


In [32]:
educ_df = pd.melt(educ_df,id_vars=key)
educ_df.columns = educ_df.columns.str.title()
educ_df.head()

,Pro_Name,Pro_Code,Bgy_Name,Bgy_Code,Variable,Value
0,Basilan,PH150700000,Arco,PH150702001,Number of School,1
1,Basilan,PH150700000,Ba-as,PH150702002,Number of School,1
2,Basilan,PH150700000,Balagtasan,PH150702005,Number of School,1
3,Basilan,PH150700000,Balas,PH150702006,Number of School,1
4,Basilan,PH150700000,Balobo,PH150702007,Number of School,1


In [33]:
educ_df.to_csv('output/educ_schools.csv',index=False)

# Water Supply

In [8]:
drink_df = pd.read_excel('data/Main Source of Water Supply for Drinking (ARMM 2015) Brgy.xlsx')
drink_df = drink_df.rename(columns={'BRGY':'Bgy_Name','PSGC':'Bgy_Code'})
drink_df = drink_df.merge(psgc,on=['Bgy_Name','Bgy_Code'],how='left')
drink_df['Usage'] = 'Drinking'
drink_df.head()

,Bgy_Name,Bgy_Code,Number of Households,Own use faucet community water system,Shared faucet community water system,Own use tubed/piped deep well,Shared tubed/piped deep well,Tubed/piped shallow well,Dug well,Protected spring,Unprotected spring,"Lake, river, rain and others",Peddler,Bottled water,Others,Pro_Name,Pro_Code,Usage
0,Arco,PH150702001,160,8,67,6,1,0,10,68,0,0,0,0,0,Basilan,PH150700000,Drinking
1,Ba-as,PH150702002,231,12,2,0,215,0,0,2,0,0,0,0,0,Basilan,PH150700000,Drinking
2,Baimbing,PH150702004,101,2,14,4,48,0,21,0,0,12,0,0,0,Basilan,PH150700000,Drinking
3,Balagtasan,PH150702005,434,43,317,13,59,0,0,0,0,2,0,0,0,Basilan,PH150700000,Drinking
4,Balas,PH150702006,546,1,7,3,207,292,5,0,0,2,0,24,5,Basilan,PH150700000,Drinking


In [9]:
cook_df = pd.read_excel('data/Main Source of Water Supply for Cooking (ARMM 2015) Brgy.xlsx')
cook_df = cook_df.rename(columns={'BRGY':'Bgy_Name','PSGC':'Bgy_Code'})
cook_df = cook_df.merge(psgc,on=['Bgy_Name','Bgy_Code'],how='left')
cook_df['Usage'] = 'Cooking'
cook_df.columns = drink_df.columns
cook_df.head()

,Bgy_Name,Bgy_Code,Number of Households,Own use faucet community water system,Shared faucet community water system,Own use tubed/piped deep well,Shared tubed/piped deep well,Tubed/piped shallow well,Dug well,Protected spring,Unprotected spring,"Lake, river, rain and others",Peddler,Bottled water,Others,Pro_Name,Pro_Code,Usage
0,Arco,PH150702001,160,8,67,6,1,0,10,68,0,0,0,0,0,Basilan,PH150700000,Cooking
1,Ba-as,PH150702002,231,11,2,0,218,0,0,0,0,0,0,0,0,Basilan,PH150700000,Cooking
2,Baimbing,PH150702004,101,4,11,4,45,2,20,2,1,12,0,0,0,Basilan,PH150700000,Cooking
3,Balagtasan,PH150702005,434,45,313,15,60,0,0,0,0,1,0,0,0,Basilan,PH150700000,Cooking
4,Balas,PH150702006,546,3,9,6,201,315,5,0,0,3,0,3,1,Basilan,PH150700000,Cooking


In [10]:
keys = key+['Usage','Number of Households']
water_df = pd.concat([drink_df,cook_df])
water_df = pd.melt(water_df,id_vars=keys)
water_df = water_df.rename(columns={'variable':'water source'})
water_df.columns = water_df.columns.str.title()
water_df.head()

,Pro_Name,Pro_Code,Bgy_Name,Bgy_Code,Usage,Number Of Households,Water Source,Value
0,Basilan,PH150700000,Arco,PH150702001,Drinking,160,Own use faucet community water system,8
1,Basilan,PH150700000,Ba-as,PH150702002,Drinking,231,Own use faucet community water system,12
2,Basilan,PH150700000,Baimbing,PH150702004,Drinking,101,Own use faucet community water system,2
3,Basilan,PH150700000,Balagtasan,PH150702005,Drinking,434,Own use faucet community water system,43
4,Basilan,PH150700000,Balas,PH150702006,Drinking,546,Own use faucet community water system,1


In [11]:
water_df.to_csv('output/water_source.csv',index=False)

## Lighting

In [12]:
light_df = pd.read_excel('data/Number of Households by Kind of Fuel for Lighting (ARMM 2015) Brgy.xlsx')
light_df = light_df.rename(columns={'BRGY':'Bgy_Name','PSGC':'Bgy_Code','Number of HH':'Number Of Households'})
light_df = light_df.merge(psgc,on=['Bgy_Name','Bgy_Code'],how='left')
light_df.head()

,Bgy_Name,Bgy_Code,Number Of Households,Electricity,Kerosene (Gaas),Liquified Petroleum Gas (LPG),Oil (vegetable animal and others),Solar panel,Solar lamp,Others,None,Not Reported,Pro_Name,Pro_Code
0,Arco,PH150702001,160,105,55,0,0,0,0,0,0,0,Basilan,PH150700000
1,Ba-as,PH150702002,231,183,48,0,0,0,0,0,0,0,Basilan,PH150700000
2,Baimbing,PH150702004,101,40,60,0,0,0,0,0,1,0,Basilan,PH150700000
3,Balagtasan,PH150702005,434,378,55,0,0,1,0,0,0,0,Basilan,PH150700000
4,Balas,PH150702006,546,393,129,1,0,0,1,22,0,0,Basilan,PH150700000


In [13]:
keys = key+['Number Of Households']
light_df = pd.melt(light_df,id_vars=keys)
light_df = light_df.rename(columns={'variable':'light source'})
light_df.columns = light_df.columns.str.title()
light_df.head()

,Pro_Name,Pro_Code,Bgy_Name,Bgy_Code,Number Of Households,Light Source,Value
0,Basilan,PH150700000,Arco,PH150702001,160,Electricity,105
1,Basilan,PH150700000,Ba-as,PH150702002,231,Electricity,183
2,Basilan,PH150700000,Baimbing,PH150702004,101,Electricity,40
3,Basilan,PH150700000,Balagtasan,PH150702005,434,Electricity,378
4,Basilan,PH150700000,Balas,PH150702006,546,Electricity,393


In [14]:
light_df.to_csv('output/light_source.csv',index=False)

## Housing

In [15]:
keys = key+['Category','Total Occupied Housing Units']

In [16]:
wall_df = pd.read_excel('data/Construction Materials of the Walls(ARMM 2015)Brgy.xlsx')
wall_df = wall_df.rename(columns={'BRGY':'Bgy_Name','PSGC':'Bgy_Code'})
wall_df = wall_df.merge(psgc,on=['Bgy_Name','Bgy_Code'],how='left')
wall_df['Category'] = 'Wall'
wall_df.head()

,Bgy_Name,Bgy_Code,Total Occupied Housing Units,Concrete/brick/stone,Wood,Half concrete/\nbrick/stone and half wood,Galvanized iron/aluminum,Bamboo/\nsawali/cogon/nipa,Asbestos,Glass,Makeshift/\nsalvaged/\nimprovised materials,Trapal,Others,No walls,Not Reported,Pro_Name,Pro_Code,Category
0,Arco,PH150702001,144,11,85,26,2,19,0,0,0,1,0,0,0,Basilan,PH150700000,Wall
1,Ba-as,PH150702002,230,57,102,39,0,32,0,0,0,0,0,0,0,Basilan,PH150700000,Wall
2,Baimbing,PH150702004,100,9,12,13,7,58,1,0,0,0,0,0,0,Basilan,PH150700000,Wall
3,Balagtasan,PH150702005,427,63,183,161,2,17,0,0,1,0,0,0,0,Basilan,PH150700000,Wall
4,Balas,PH150702006,546,32,162,47,5,293,0,0,0,0,0,0,7,Basilan,PH150700000,Wall


In [17]:
wall_df = pd.melt(wall_df,id_vars=keys)
wall_df = wall_df.rename(columns={'variable':'material'})
wall_df.columns = wall_df.columns.str.title()
wall_df.head()

,Pro_Name,Pro_Code,Bgy_Name,Bgy_Code,Category,Total Occupied Housing Units,Material,Value
0,Basilan,PH150700000,Arco,PH150702001,Wall,144,Concrete/brick/stone,11
1,Basilan,PH150700000,Ba-as,PH150702002,Wall,230,Concrete/brick/stone,57
2,Basilan,PH150700000,Baimbing,PH150702004,Wall,100,Concrete/brick/stone,9
3,Basilan,PH150700000,Balagtasan,PH150702005,Wall,427,Concrete/brick/stone,63
4,Basilan,PH150700000,Balas,PH150702006,Wall,546,Concrete/brick/stone,32


In [18]:
roof_df = pd.read_excel('data/Construction Materials of the Roof (ARMM 2015)Brgy.xlsx')
roof_df = roof_df.rename(columns={'BRGY':'Bgy_Name','PSGC':'Bgy_Code'})
roof_df = roof_df.merge(psgc,on=['Bgy_Name','Bgy_Code'],how='left')
roof_df['Category'] = 'Roof'
roof_df.head()

,Bgy_Name,Bgy_Code,Total Occupied Housing Units,Galvanized iron/\naluminum,Tile/concrete/\nclay tile,Half galvanized iron and half concrete,Bamboo/\ncogon/ nipa/anahaw,Asbestos,Makeshift/ salvaged/ improvised materials,Trapal,Others,Not Reported,Pro_Name,Pro_Code,Category
0,Arco,PH150702001,144,97,1,1,44,0,0,1,0,0,Basilan,PH150700000,Roof
1,Ba-as,PH150702002,230,203,0,3,24,0,0,0,0,0,Basilan,PH150700000,Roof
2,Baimbing,PH150702004,100,20,2,10,68,0,0,0,0,0,Basilan,PH150700000,Roof
3,Balagtasan,PH150702005,427,16,55,188,167,0,1,0,0,0,Basilan,PH150700000,Roof
4,Balas,PH150702006,546,371,0,4,168,0,3,0,0,0,Basilan,PH150700000,Roof


In [19]:
roof_df = pd.melt(roof_df,id_vars=keys)
roof_df = roof_df.rename(columns={'variable':'material'})
roof_df.columns = roof_df.columns.str.title()
roof_df.head()

,Pro_Name,Pro_Code,Bgy_Name,Bgy_Code,Category,Total Occupied Housing Units,Material,Value
0,Basilan,PH150700000,Arco,PH150702001,Roof,144,Galvanized iron/\naluminum,97
1,Basilan,PH150700000,Ba-as,PH150702002,Roof,230,Galvanized iron/\naluminum,203
2,Basilan,PH150700000,Baimbing,PH150702004,Roof,100,Galvanized iron/\naluminum,20
3,Basilan,PH150700000,Balagtasan,PH150702005,Roof,427,Galvanized iron/\naluminum,16
4,Basilan,PH150700000,Balas,PH150702006,Roof,546,Galvanized iron/\naluminum,371


In [20]:
house_df = pd.concat([wall_df,roof_df])
house_df.head()

,Pro_Name,Pro_Code,Bgy_Name,Bgy_Code,Category,Total Occupied Housing Units,Material,Value
0,Basilan,PH150700000,Arco,PH150702001,Wall,144,Concrete/brick/stone,11
1,Basilan,PH150700000,Ba-as,PH150702002,Wall,230,Concrete/brick/stone,57
2,Basilan,PH150700000,Baimbing,PH150702004,Wall,100,Concrete/brick/stone,9
3,Basilan,PH150700000,Balagtasan,PH150702005,Wall,427,Concrete/brick/stone,63
4,Basilan,PH150700000,Balas,PH150702006,Wall,546,Concrete/brick/stone,32


In [21]:
house_df['Material'].value_counts()

Others                                         4980
Not Reported                                   4980
Trapal                                         4980
Asbestos                                       4980
Galvanized iron/\naluminum                     2490
Bamboo/\nsawali/cogon/nipa                     2490
Concrete/brick/stone                           2490
Makeshift/\nsalvaged/\nimprovised materials    2490
Tile/concrete/\nclay tile                      2490
Galvanized iron/aluminum                       2490
Wood                                           2490
Half galvanized iron and half concrete         2490
Half concrete/\nbrick/stone and half wood      2490
Makeshift/ salvaged/ improvised materials      2490
No walls                                       2490
Glass                                          2490
Bamboo/\ncogon/ nipa/anahaw                    2490
Name: Material, dtype: int64

In [22]:
house_df.to_csv('output/housing.csv',index=False)

## Work / Occupation

In [23]:
work_df = pd.read_excel('data/Gainful Workers 15 Years Old and Over by Major Occupation Group, Age Group, Sex (ARMM 2015) Barangay.xlsx')
work_df = work_df.rename(columns={'Brgy':'Bgy_Name','PSGC':'Bgy_Code','Number of HH':'Number Of Households'})
work_df = work_df.merge(psgc,on=['Bgy_Name','Bgy_Code'],how='left')
work_df.head()

,Bgy_Name,Bgy_Code,tot_15_ovr_both_sexes,tot_15 - 19_both_sexes,tot_20 - 24_both_sexes,tot_25 - 29_both_sexes,tot_30 - 34_both_sexes,tot_35 - 39_both_sexes,tot_40 - 44_both_sexes,tot_45 - 49_both_sexes,...,not_30 - 34_fem,not_35 - 39_fem,not_40 - 44_fem,not_45 - 49_fem,not_50 - 54_fem,not_55 - 59_fem,not_60 - 64.2,not_65 and over_fem,Pro_Name,Pro_Code
0,Arco,PH150702001,211,10,37,33,25,23,20,20,...,0,0,0,0,0,0,0,0,Basilan,PH150700000
1,Ba-as,PH150702002,336,22,48,55,48,36,39,24,...,0,0,0,0,0,0,0,0,Basilan,PH150700000
2,Baimbing,PH150702004,149,5,19,18,16,15,19,18,...,0,0,0,0,0,0,0,0,Basilan,PH150700000
3,Balagtasan,PH150702005,794,41,101,118,120,90,64,83,...,0,0,0,0,0,0,0,0,Basilan,PH150700000
4,Balas,PH150702006,1002,102,156,149,129,88,96,85,...,0,0,0,0,0,0,0,0,Basilan,PH150700000


In [24]:
total_cols = [col for col in work_df.columns if '_15_ovr_both' in col]
total_cols.remove('tot_15_ovr_both_sexes')
keys = key+['tot_15_ovr_both_sexes']
work_df = pd.melt(work_df,id_vars=keys,value_vars=total_cols)
work_df = work_df.rename(columns={'variable':'occupation type'})
work_df.columns = work_df.columns.str.title()
work_df.head()

,Pro_Name,Pro_Code,Bgy_Name,Bgy_Code,Tot_15_Ovr_Both_Sexes,Occupation Type,Value
0,Basilan,PH150700000,Arco,PH150702001,211,man_15_ovr_both_sexes,4
1,Basilan,PH150700000,Ba-as,PH150702002,336,man_15_ovr_both_sexes,25
2,Basilan,PH150700000,Baimbing,PH150702004,149,man_15_ovr_both_sexes,14
3,Basilan,PH150700000,Balagtasan,PH150702005,794,man_15_ovr_both_sexes,43
4,Basilan,PH150700000,Balas,PH150702006,1002,man_15_ovr_both_sexes,77


In [25]:
occ_dict={'man': 'Managers',
 'pro': 'Professionals',
 'tec': 'Technicians and Associate Professionals',
 'cle': 'Clerical Support Workers',
 'ser': 'Service and Sales Workers',
 'ski': 'Skilled Agricultural Forestry and Fishery Workers',
 'cra': 'Craft and Related Trades Workers',
 'pla': 'Plant and Machine Operators and Assemblers',
 'ele': 'Elementary Occupations',
 'arm': 'Armed Forces Occupations',
 'oth': 'Other Occupation Not Elsewhere Classified',
 'not': 'Not Reported'}
work_df['Occupation Type'] = work_df['Occupation Type'].str.split('_',1).str[0]
work_df['Occupation Type'] = work_df['Occupation Type'].replace(occ_dict)
work_df['Occupation Type'].unique()

array(['Managers', 'Professionals',
       'Technicians and Associate Professionals',
       'Clerical Support Workers', 'Service and Sales Workers',
       'Skilled Agricultural Forestry and Fishery Workers',
       'Craft and Related Trades Workers',
       'Plant and Machine Operators and Assemblers',
       'Elementary Occupations', 'Armed Forces Occupations',
       'Other Occupation Not Elsewhere Classified'], dtype=object)

In [26]:
work_df.to_csv('output/work_type.csv',index=False)